In [1]:
if(!dir.exists("Functions/")){
    setwd("../")
    if(!dir.exists("Functions")){
        setwd("M:/lecospec/lecospec/")
    }
}
source("Functions/lecospectR.R", echo = FALSE)

Loading required package: tidyverse

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: compiler

Loading required package: raster

Loading required package: sp


Attaching package: ‘raster’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: hsdar

Loading required package: rgdal

Please note that rgdal will be retired during 2023,
plan transition to sf/stars/terra functions using GDAL and PROJ
at your earliest convenience.
See https://r-spatial.org/r/2

In [2]:
log_model_results <- function(model_id, confusion_matrix, distribition, custom = NULL, logpath = "./gs.log"){
    # append performance data to the logs for later comparison
    sink(file = logpath, append = TRUE)
    print("-------------------------------------------------------")
    print("---------------------- Model Data ---------------------")
    
    print(paste0("Model Type: PLS-LDA (plsgenomics)"))
    print(paste0("Data Index: ",custom))
    print(paste0("Model UUID: ", model_id))
    print("---------------------- Confusion Matrix ---------------------")
    print(confusion_matrix)
    print("---------------------- Class Distribution ---------------------")
    print(distribition)
    print("-------------------------------------------------------")
    sink(NULL)
}

In [3]:
add_model_to_manifest <- function(
    model_id, 
    outlier = "", 
    preprocessing="",
    source="", 
    weight = "",
    n = "",
    oob_error = "",
    accuracy = "",
    r2 = "",
    chi2prob = "",
    logpath="./gs_manifest.csv"){
    if(!file.exists(logpath)){
        header <- "source,outliers,preprocessing,weight,n,oob,accuracy,r2,chi2prob,model_id"
        write(header, file = logpath)
    }

    line <- paste(
        source,
        outlier,
        preprocessing,
        weight,
        n,
        oob_error,
        accuracy,
        r2,
        chi2prob,
        sep=","
    )
    line <- paste0(line, ",", model_id)

    write(line, file=logpath, append = TRUE)
}

In [4]:
train_pls_lda <- function(
    train_df, 
    train_labels,
    test_df, 
    test_labels,
    n = 32,
    outlier_fn = NULL,
    preprocess_fn = NULL,
    weight_fn = targets_to_weights,
    model_id = uuid::UUIDgenerate(),
    ignore_cols = NULL,
    save_path = "./mle/models/gs/",
    seed = NULL,
    log_string = ""
){

    if(!is.null(seed)){
        set.seed(seed)
    }

    x_train <- train_df %>% as.data.frame()
    x_test <- test_df %>% as.data.frame()
    if(is.function(outlier_fn)){
        x_train <- outlier_fn(x_train)
    }
    if(is.function(preprocess_fn)){
        x_train <- preprocess_fn(x_train)
        x_test <- preprocess_fn(x_test)
    }
    # nrounds, max_depth, eta, gamma, colsample_bytree, min_child_weight, subsample
    grid <- expand.grid(
        nrounds=1000,
        max_depth = c(32, 64),
        eta = c(0.05, 0.01),
        gamma = c(0.5, 1),
        colsample_bytree = c(0.6, 0.8),
        min_child_weight = c(1, 2, 4),
        subsample = c(0.3, 0.4)
    )

    if(("Forb" %in% levels(train_labels)) && !("Forb"  %in% levels(test_labels))){
        levels(test_labels) <- c(levels(test_labels), "Forb")
        }
        
    train_ctrl <- caret::trainControl(
        method = "repeatedcv",
        number = 10,
        search = "random",
        sampling = NULL,
        repeats = 25
    )
    
    pls_model <- caret::train(
        x_train, 
        train_labels, 
        #maxit = 100000,
        method="xgbTree",
        preProcess = c("center", "scale", "knnImpute"),
        tree_method = "hist",
        weights = weight_fn(train_labels),
        trControl = train_ctrl
        #tuneGrid = grid
        #tuneLength = n
    )

    print(pls_model)

    save(
        pls_model,
        file = file.path(save_path, paste0(model_id, ".rda"))
    )

    # create predictions (ranger)
    model_predictions <- predict(
            pls_model, 
            newdata = x_test
        )

    print(model_predictions)


    # generate the confusion matrix

    confusion_matrix <- caret::confusionMatrix(
        test_labels,
        model_predictions %>% as.factor(),
        mode = "everything"
    )

    log_model_results(
            model_id = model_id,
            confusion_matrix = confusion_matrix,
            custom = log_string,
            distribition = model_predictions %>% as.factor() %>% table(),
            logpath = "./gs_xgb.log")

    return(
        list(
                model = pls_model,
                confusion = confusion_matrix %>% as.list()
                )
    )

    

}

In [5]:
base_paths <- c(
    "img_raw_raw.csv",
    #"grd_raw_raw.csv",
    "corrected_and_img.csv"
    #"grd_corrected_only_bands.csv",# include veg indices
    #"img_only_bands.csv",
    #"grd_only_bands.csv",
    #"corrected_and_img_only_bands.csv",
    #"img_indices_only.csv"# include veg indices
    #"grd_raw_corrected.csv",
    #"grd_indices_only.csv"
)
# 

In [6]:
calculate_posterior_weights <- function(validation_path ="figures/merged_validation_s.csv" ){

    validation_df <- read.csv(validation_path, header = TRUE)
    #print(head(validation_df))

    total_observations <- sum(validation_df$validation_counts)
    #print(total_observations)
    weights <- (1/ validation_df$validation_prop)
    #print(validation_df$validation_prop)

    total_by_fg1 <- aggregate(
        x = validation_df$validation_counts,
        by = list(validation_df$key),
        FUN = sum
    )

    fg1_weight_list <- list()

    for( row_idx in seq(nrow(total_by_fg1))){
        name <- total_by_fg1$Group.1[[row_idx]]
        value <- total_by_fg1$x[[row_idx]]
        fg1_weight_list[name] <- value
    }
    
    return(fg1_weight_list)
}

get_posterior_weights_from_targets <- function(target_factor, posterior_weight = calculate_posterior_weights()){
    unbiased_weights <- targets_to_weights(target_factor)

    target_name_char <- target_factor %>% as.character()

    output_weights <- seq_along(target_factor)

    for(i in seq_along(target_factor)){
        if(posterior_weight[[target_name_char[[i]]]] > 0){
            fg1_weight <- 1 / posterior_weight[[target_name_char[[i]]]]
        } else {
            fg1_weight <- 0
        }
        output_weights[[i]] <- unbiased_weights[[i]] * fg1_weight
    }

    return(output_weights)
}

In [7]:
filter_target_subset <- function(df){

    df_cols <- colnames(df) %>% as.character()
    target_cols <- c(
        "Vogelmann",
        "TVI",
        "CRI4",
        "Datt5",
        "DWSI4",
        "GDVI",
        "MCARI",
        "MTVI",
        "NPCI",
        "PARS",
        "X417.593_5nm",
        "X787.593_5nm",
        "X892.593_5nm"
    )

    used_cols <- intersect(df_cols, target_cols)
    print(paste0("Filtering Data frame to the following columns: ", used_cols))
    return(df[,used_cols])
}

In [8]:
filter_features <- function(df){

    df_cols <- colnames(df) %>% as.character()
    target_cols <- c(
        "TCARIOSAVI",
        "TCARIO2SAVI2",
        "Vogelmann3",
        "Vogelmann4",
        "CI",
        "Carter",
        "Carter2",
        "Carter3",
        "Carter4",
        "Carter5",
        "Carter6",
        "D1",
        "D2",
        "Datt",
        "Datt3",
        "Datt5",
        "DPI",
        "EVI",
        "Gittelson",
        "PARS",
        "Maccioni",
        "mSR",
        "mSR705",
        "MTCI",
        "PRInorm",
        "REPLE",
        "REPLi",
        "SIPI"
    )

    used_cols <- setdiff(df_cols, target_cols)
    print(paste0("Removed the following columns: ", 
    intersect(df_cols, target_cols)))
    return(df[,used_cols])
}

## Hypotheses

Peter
* Increasing number of trees will improve the accuracy/validation mismatch
* bias in 

Ken
* decreasing the number of the trees will decrease the impact of the posterior weighting on chi-squared statistic

In [9]:
outlier_functions <- list(
    #clip = load_model("./mle/clip_transform.rda"),
    no_treatment = function(x, ignore_cols = NULL){return(x)}# no transform
)

outlier_treatments <- c(
    "no_treatment",
    "clip"
)

preprocess_functions <- list(
    no_treatment = function(x, ignore_cols = NULL){return(x)},# no transform
    min_max = columnwise_min_max_scale,
    robust = columnwise_robust_scale,
    standard = standardize_df
)

weight_functions <- list(
    posterior = get_posterior_weights_from_targets,
    balanced = targets_to_weights,
    no_treatment = function(x){return(NULL)}# No weights
)
weight_treatments <- c(
    "balanced"#,
    #"no_treatment",
    #"posterior"
)

preprocessing_treatments <- c(
    "no_treatment"#,
    #"standard",
    #"min_max",
    #"robust"
)

In [10]:
num_components <- c(
    1,2,4,6,8,10,12,14,16,18,20,24,32,50#,64,75,100,128,200,256,512,1000
)

In [11]:
test_data <- subset(read.csv("Data/gs/x_test/img_raw_raw.csv"), select = -c(X))
test_labels <- read.csv("Data/gs/y_test/img_raw_raw.csv")$x %>% as.factor()
#train_labels <- read.csv("Data/gs/y_train/img_raw_raw.csv")$x %>% as.factor()
manifest_path <- "./gs_manifest_xgboost_fg0.csv"

In [12]:
for(filepath in base_paths){
    train_data <- subset(read.csv(paste0("Data/gs/x_train/", filepath)), select = -c(X))
    labels <- read.csv(paste0("Data/gs/y_train/", filepath))$x %>% as.factor()

                n <- "NA"
                p_treatment <- "no_treatment"
                o_treatment <- "no_treatment"
                w_treatment <- "no_treatment"
                print(p_treatment)
                print("Number of columns in train and test:")
                print(ncol(train_data))
                print(ncol(test_data))
                print("Number of rows in train and number of labels:")
                print(nrow(train_data))
                print(length(labels))

                model_id <- uuid::UUIDgenerate()
                save_path <- paste0("mle/experiments/gs/", model_id, "/")
                if(!dir.exists(save_path)){
                    dir.create(save_path)
                }

                rf_model_results <- train_pls_lda(
                    filter_features(train_data), 
                    labels, 
                    test_data,
                    test_labels,
                    n = n,
                    outlier_fn = outlier_functions[[o_treatment]],
                    preprocess_fn = preprocess_functions[[p_treatment]],
                    weight_fn = weight_functions[[w_treatment]],
                    model_id = model_id,
                    seed=61718,
                    log_string = paste(n, filepath, o_treatment, p_treatment, w_treatment)
                )
                print(rf_model_results)

                rf_model <- rf_model_results$model
                acc <- as.list(rf_model_results$confusion$overall)$Accuracy
                print(acc)

                if(acc > 0.6){

                results <- validate_model(
                    rf_model, 
                    save_path, 
                    outlier_processing = outlier_functions[[o_treatment]],
                    transform_type = preprocess_functions[[p_treatment]],
                    pft_aggregation=0
                )

                aggregated_results <- aggregate_results(save_path)

                # calculate validation statistics
                chi2 <- calculate_chi_squared_probability(aggregated_results)
                r2 <- calculate_validation_r2(aggregated_results)
                rpd <- calculate_rpd(aggregated_results)


                add_model_to_manifest(
                    model_id = model_id,
                    outlier = o_treatment,
                    preprocessing = p_treatment,
                    source = filepath,
                    weight = w_treatment,
                    n = n,
                    oob_error = "NA",
                    accuracy = acc,
                    r2 = r2,
                    chi2prob = rpd,
                    logpath=manifest_path
                )

                plot_by_pft(
                    aggregated_results,
                    save_path = paste0(save_path, "aggregate.html"),
                    open = FALSE,
                    image_path = NULL,
                    aggregation=0
                )
            #
                write_validation_table(
                    aggregated_results,
                    save_path = paste0(save_path, "table.html"),
                    open = FALSE
                )
                } else {
                    add_model_to_manifest(
                        model_id = model_id,
                        outlier = o_treatment,
                        preprocessing = p_treatment,
                        source = filepath,
                        weight = w_treatment,
                        n = n,
                        oob_error = rf_model$prediction.error,
                        accuracy = acc,
                        r2 = "Skipped",
                        chi2prob = "Skipped",
                        logpath=manifest_path
                    )
                }
 }

[1] "no_treatment"
[1] "Number of columns in train and test:"
[1] 214
[1] 214
[1] "Number of rows in train and number of labels:"
[1] 4681
[1] 4681
 [1] "Filtering Data frame to the following columns: X417.593_5nm"
 [2] "Filtering Data frame to the following columns: X787.593_5nm"
 [3] "Filtering Data frame to the following columns: X892.593_5nm"
 [4] "Filtering Data frame to the following columns: CRI4"        
 [5] "Filtering Data frame to the following columns: Datt5"       
 [6] "Filtering Data frame to the following columns: DWSI4"       
 [7] "Filtering Data frame to the following columns: MCARI"       
 [8] "Filtering Data frame to the following columns: MTVI"        
 [9] "Filtering Data frame to the following columns: NPCI"        
[10] "Filtering Data frame to the following columns: PARS"        
[11] "Filtering Data frame to the following columns: TVI"         
[12] "Filtering Data frame to the following columns: Vogelmann"   
eXtreme Gradient Boosting 

4681 samples
  12 pr

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.

`.cols` has been renamed and is deprecated, please use `.vars`
Warning message:
“`funs()` was deprecated in dplyr 0.8.0.
ℹ Please use a list of either functions or lambdas:

# Simple named list: list(mean = mean, median = median)

# Auto named with `tibble::lst()`: tibble::lst(mean, median)

# Using lambdas list(~ mean(., trim = .2), ~ median(., na.rm = TRUE))
ℹ The deprecated feature was likely used in the useful package.
  Please report the issue at <https://github.com/jaredlander/useful/issues>.”


[1] "Handling Outliers with User supplied function"
[1] "Transforming Data with user supplied functions"


`.cols` has been renamed and is deprecated, please use `.vars`


Reading layer `Bisoon_Quadrats_georeferenced' from data source 
  `/home/krbundy/GitHub/lecospec/Data/Vectors/Bisoon_Quadrats_georeferenced.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 9 features and 3 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -148.9524 ymin: 63.80698 xmax: -148.9508 ymax: 63.80701
Geodetic CRS:  WGS 84


...

`.cols` has been renamed and is deprecated, please use `.vars`


Reading layer `Bonanza_N_Quads' from data source 
  `/home/krbundy/GitHub/lecospec/Data/Vectors/Bonanza_N_Quads.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 11 features and 3 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: 438153.7 ymin: 7176051 xmax: 438205.6 ymax: 7176138
Projected CRS: WGS 84 / UTM zone 6N


Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Warning message in chisq.test(validation, prediction):
“Chi-squared approximation may be incorrect”


[1] "no_treatment"
[1] "Number of columns in train and test:"
[1] 95
[1] 214
[1] "Number of rows in train and number of labels:"
[1] 3876
[1] 4682
[1] "Filtering Data frame to the following columns: CRI4"     
[2] "Filtering Data frame to the following columns: Datt5"    
[3] "Filtering Data frame to the following columns: DWSI4"    
[4] "Filtering Data frame to the following columns: MCARI"    
[5] "Filtering Data frame to the following columns: MTVI"     
[6] "Filtering Data frame to the following columns: NPCI"     
[7] "Filtering Data frame to the following columns: PARS"     
[8] "Filtering Data frame to the following columns: TVI"      
[9] "Filtering Data frame to the following columns: Vogelmann"


ERROR: Error in check_dims(x = x, y = y): nrow(x) == n is not TRUE
